# Plotting for experiments 1 and 2

## TODO: Get this to work properly

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import graphlearning as gl
import graphlearning.active_learning as al

import batch_active_learning as bal
import utils

from matplotlib.ticker import AutoLocator

import pickle5 as pickle

In [ ]:
#Non-default Parameters
acq_fun_list = ['uc']
save_path = "Experiment Results/Experiment 1/"

In [ ]:
def tick_function(X, n):
    V = X / n
    return ["%d%%" % np.round(100*z) for z in V]
    
def experiment1_plotter(x_dict, y_dict, dataset, include_sota=False, fine_tuned=False):
    fig = plt.figure()
    fig.set_size_inches(12.5, 8.5)
    ax1 = fig.add_subplot()
    ax2 = ax1.twiny()

    dataset_size = utils.SAR_DATASET_SIZE_DICT[dataset]

    ax1.set_xlabel('Number of Labeled Points')
    ax1.set_ylabel('Accuracy (%)')
    
    marker_types = ["^", "o", "d", "p", "*"]
    for i in range(len(bal.AL_METHODS)):
        this_key = bal.AL_METHODS[i]
        ax1.plot(x_dict[this_key], y_dict[this_key], label=bal.AL_METHOD_NAMES[i], linewidth=3, markevery=3, markersize=15)

    ax1.tick_params(axis='x')

    # Add SoTA
    if dataset != 'mstar' and include_sota:
        if dataset == 'open_sar_ship':
            sota_val = 78.15
        elif dataset == 'fusar':
            sota_val = 86.69
        ax1.plot(x_dict['local_max'], sota_val *
                 np.ones_like(x_dict['local_max']), label='SoTA', linestyle='--')

    fontsize = 30
    #ax1.legend(fontsize=fontsize)
    ax1.set_xlabel(r"Number of labeled points", fontsize=fontsize)
    ax1.set_ylabel("Accuracy (%)", fontsize=fontsize)
    ax1.set_ylim((None, 100))
    if fine_tuned:
        ax1.set_xlim((dataset_size*0.05, None))
    else:
        ax1.set_xlim((0, None))
    
    ax1.autoscale(enable=True, axis='x', tight=False)

    
    
    if fine_tuned:
        if dataset == 'mstar':
            new_tick_locations = np.linspace(0.05, 0.15, 5) * dataset_size
        elif dataset == 'open_sar_ship':
            new_tick_locations = np.linspace(0.05, 0.3, 5) * dataset_size
        else:
            new_tick_locations = np.linspace(0.05, 0.63, 6) * dataset_size
    else:
        if dataset == 'mstar':
            new_tick_locations = np.linspace(0, 0.15, 5) * dataset_size
        elif dataset == 'open_sar_ship':
            new_tick_locations = np.linspace(0, 0.3, 5) * dataset_size
        else:
            new_tick_locations = np.linspace(0, 0.63, 6) * dataset_size

    
    ax2.set_xlim(ax1.get_xlim())
    ax2.set_xticks(new_tick_locations)
    ax2.set_xticklabels(tick_function(new_tick_locations, dataset_size), fontsize=fontsize)
    ax2.set_xlabel(r"Percentage of Training Data", fontsize=fontsize)
    
    ax1.tick_params(axis='both', labelsize=fontsize)
    ax2.tick_params(axis='both', labelsize=fontsize)

    return

def get_data(dataset, file):
    return pd.read_pickle(save_path + 'Pickles/' + dataset + "_" + file + '.pkl')

In [ ]:
dataset = 'fusar'
df = get_data(dataset, 'full_acc_dict')
df = df.T

In [ ]:
print(df)

In [ ]:
y_dict = {'local_max': [n for n in df['local_max'] if not np.isnan(n)],
          'random': [n for n in df['random'] if not np.isnan(n)],
          'topn_max': [n for n in df['topn_max'] if not np.isnan(n)],
          'acq_sample': [n for n in df['acq_sample'] if not np.isnan(n)],
          'global_max': [n for n in df['global_max'] if not np.isnan(n)]}
x_dict = {'local_max': [n*15 for n in range(len(y_dict['local_max']))],
          'random': [n*15 for n in range(len(y_dict['random']))],
          'topn_max': [n*15 for n in range(len(y_dict['topn_max']))],
          'acq_sample': [n*15 for n in range(len(y_dict['acq_sample']))],
          'global_max': [n for n in range(len(y_dict['global_max']))]}

In [ ]:
print(len(x_dict['local_max']))
print(len(y_dict['local_max']))

In [ ]:
experiment1_plotter(x_dict, y_dict, dataset, include_sota=True)